<a href="https://colab.research.google.com/github/nekoniii3/openai_3syu/blob/main/sample/Whisper_%E9%9F%B3%E5%A3%B0%E5%85%A5%E5%8A%9B_%E5%9F%BA%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio==4.2.0
!pip install openai==1.3.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 5.7 MB/s e

In [2]:
import gradio as gr
from openai import OpenAI
import os

audio_format = [".mp3", ".mp4", ".mpeg", ".mpga", ".m4a", ".wav", ".webm"
                ,".MP3", ".MP4", ".MPEG", ".MPGA", ".M4A", ".WAV", ".WEBM"]

lang_code = {'Japanese': "ja", 'English': "en"}
mode_code = {'文字': "transcribe", '翻訳': "translate"}

max_file_size = 25 * 1024 * 1024

def set_state(openai_key, lang, mode, state):

    state["openai_key"]= openai_key
    state["lang"] = lang_code[lang]
    state["mode"] = mode_code[mode[0:2]]

    return state

def create_textfile(voice_msg, up_file ,state):

    # OpenAIキーチェック
    if state["openai_key"] is None or state["openai_key"] == "":

        err_msg = "OpenAIキーを入力してください。（設定タブ）"

        if up_file:
            return voice_msg, up_file.name, "", None, err_msg

        return voice_msg, None, "", None, err_msg

    # ファイル入力チェック
    if up_file is None and voice_msg is None:

        err_msg = "マイクかファイルで音声を入力してください。"

        return None, None, "", None, err_msg

    # アップロードの場合ファイルチェック
    if voice_msg is None:

        # ファイルパスなど取得
        file_name = os.path.splitext(os.path.basename(up_file.name))[0]
        root, ext = os.path.splitext(up_file.name)

        if ext not in audio_format:

            # ファイル形式チェック
            err_msg = "指定した形式のファイルをアップしてください。（mp3, mp4, mpeg, mpga, m4a, wav, webm）"

            return None, None, "", None, err_msg

        file_size = os.path.getsize(up_file.name)

        if file_size >= max_file_size:

            # ファイルサイズチェック
            err_msg = "ファイルが大きすぎます。25MB未満にして下さい。"

            return None, up_file.name, "", None, err_msg

    # 音声ファイルセット
    if voice_msg:
        audio_file= open(voice_msg, "rb")

    else:
        audio_file= open(up_file.name, "rb")

    try:

      # whisperで変換処理
      os.environ["OPENAI_API_KEY"] = os.environ["TEST_OPENAI_KEY"]   # テスト時
      # os.environ["OPENAI_API_KEY"] = state["openai_key"]

      # クライアント新規作成
      client = OpenAI()

      # client作成後は消す
      os.environ["OPENAI_API_KEY"] = ""

      if state["mode"] == "transcribe":

          # 文字起こし実行
          transcript = client.audio.transcriptions.create("whisper-1", audio_file ,language = state["lang"])

      else:

          # 翻訳文字起こし実行
          transcript = client.audio.transcriptions.create("whisper-1", audio_file)

      trans_text = str(transcript["text"])

      # ファイル名設定
      if voice_msg:
          trans_file = trans_text[:8] + "_whisper.txt"
      else:
          trans_file = file_name + "_whisper.txt"

      with open(trans_file, mode="w") as f:

          # テキストに書き出す
          f.write(trans_text)

      return None, None, trans_text, trans_file, ""

    except Exception as e:

      return voice_msg, None, "", None, e


with gr.Blocks() as demo:

    gr.Markdown("設定タブでOpenAIキーを入力してください。")

    # セッションの宣言
    state = gr.State({
        "openai_key" : "",
        "lang": "",
        "mode" : "",
    })

    with gr.Tab("whisperを利用する") as maintab:

      # 各コンポーネント定義
      voice_msg=gr.components.Audio(sources="microphone",type="filepath", label="音声入力")
      up_file = gr.File(file_types=[".mp3", ".mp4", ".mpeg", ".mpga", ".m4a", ".wav", ".webm"], label="音声ファイルアップロード")

      # ボタン類
      with gr.Row():
        btn = gr.Button("文字に起こす")
        clear = gr.ClearButton([voice_msg, up_file])

      # 出力
      sys_msg = gr.Text(label="システムメッセージ")
      text = gr.TextArea(label="文字起こし内容")
      file = gr.File(label="出力テキストファイル")

      # 送信ボタンクリック時の処理
      btn.click(create_textfile, inputs=[voice_msg, up_file, state], outputs=[voice_msg, up_file, text, file, sys_msg], queue=False)

    with gr.Tab("設定") as settab:
      openai_key = gr.Textbox(label="OpenAI API Key" ,type='password', interactive = True)
      lang = gr.Dropdown(label="Language", choices=["Japanese", "English"], value = "Japanese", interactive = True)
      mode = gr.Dropdown(label="Mode", choices=["文字起こし", "翻訳＋文字起こし"], value = "文字起こし", interactive = True)

      # 設定変更時
      maintab.select(set_state, [openai_key, lang, mode, state], state)

    with gr.Tab("利用上の注意"):

      gr.Markdown("・マイクの音声とファイル入力の両方がある場合はマイクが優先されます。")
      gr.Markdown("・文字に起こせるファイル形式は mp3, mp4, mpeg, mpga, m4a, wav, webm のみです。")
      gr.Markdown("・翻訳は日本語の音声を英語のテキストに変えます。英語の音声→日本語はできません。")
      gr.Markdown("※翻訳機能はマイク入力の精度が低いです。ファイルでの入力をおすすめします。")

demo.queue()
demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://ad1d364e22f333f498.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://ad1d364e22f333f498.gradio.live
